In [1]:
print('hello')

hello


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, utils

from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from tqdm import tqdm

In [ ]:
# # import os
# # import pandas as pd

# # def load_dataset_as_dataframe(subdir):
# #     # Define the path to the dataset
# #     data_dir = f'e:\\Bahria University\\Open-CV lab\\Project\\{subdir}'
# #     print(f'Load dataset from `{subdir}` subdirectory')

# #     # Create lists to store image paths and labels
# #     image_paths = []
# #     labels = []

# #     # Get the list of class directories
# #     classes = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
# #     classes.sort()
# #     print('Classes:', classes)

# #     # Map class names to labels
# #     class_to_idx = {class_name: idx for idx, class_name in enumerate(classes)}

# #     # Loop over each class directory
# #     for class_name in classes:
# #         class_dir = os.path.join(data_dir, class_name)
# #         label = class_to_idx[class_name]
# #         # Get all image files in the class directory
# #         for img_name in os.listdir(class_dir):
# #             img_path = os.path.join(class_dir, img_name)
# #             if os.path.isfile(img_path):
# #                 image_paths.append(img_path)
# #                 labels.append(label)

# #     # Create a DataFrame
# #     data = pd.DataFrame({
# #         'image_path': image_paths,
# #         'label': labels
# #     })

# #     # Show the distribution of labels in the dataset
# #     dataset_distribution_dict = {i: len(data[data['label'] == i]) for i in range(len(classes))}
# #     print(dataset_distribution_dict)
# #     print()

# #     return data

# import os
# import pandas as pd

# def load_dataset_as_dataframe(subdir, max_images=100):
#     # Define the path to the dataset
#     data_dir = f'e:\\Bahria University\\Open-CV lab\\Project\\{subdir}'
#     print(f'Load dataset from `{subdir}` subdirectory')

#     # Create lists to store image paths and labels
#     image_paths = []
#     labels = []

#     # Get the list of class directories
#     classes = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
#     classes.sort()
#     print('Classes:', classes)

#     # Map class names to labels
#     class_to_idx = {class_name: idx for idx, class_name in enumerate(classes)}

#     # Counter to track the number of images processed
#     count = 0

#     # Loop over each class directory
#     for class_name in classes:
#         class_dir = os.path.join(data_dir, class_name)
#         label = class_to_idx[class_name]
#         # Get all image files in the class directory
#         for img_name in os.listdir(class_dir):
#             if count >= max_images:
#                 break  # Stop if the maximum number of images is reached
#             img_path = os.path.join(class_dir, img_name)
#             if os.path.isfile(img_path):
#                 image_paths.append(img_path)
#                 labels.append(label)
#                 count += 1
#         if count >= max_images:
#             break  # Stop processing if the maximum number of images is reached

#     # Create a DataFrame
#     data = pd.DataFrame({
#         'image_path': image_paths,
#         'label': labels
#     })

#     # Show the distribution of labels in the dataset
#     dataset_distribution_dict = {i: len(data[data['label'] == i]) for i in range(len(classes))}
#     print(dataset_distribution_dict)
#     print()

#     return data



In [ ]:
def load_dataset_as_dataframe(subdir, max_images=100):
    # Define the path to the dataset
    data_dir = f'e:\\Bahria University\\Open-CV lab\\Project\\{subdir}'
    print(f'Loading dataset from `{subdir}` subdirectory')

    # Create lists to store image paths and labels
    image_paths = []
    labels = []

    # Get the list of class directories
    classes = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
    classes.sort()
    print('Classes:', classes)

    # Map class names to labels
    class_to_idx = {class_name: idx for idx, class_name in enumerate(classes)}

    # Counter to track the number of images processed
    count = 0

    # Loop over each class directory
    for class_name in classes:
        class_dir = os.path.join(data_dir, class_name)
        label = class_to_idx[class_name]
        # Get all image files in the class directory
        for img_name in os.listdir(class_dir):
            if count >= max_images:
                break  # Stop if the maximum number of images is reached
            img_path = os.path.join(class_dir, img_name)
            if os.path.isfile(img_path):
                image_paths.append(img_path)
                labels.append(label)
                count += 1
        if count >= max_images:
            break  # Stop processing if the maximum number of images is reached

    # Create a DataFrame
    data = pd.DataFrame({
        'image_path': image_paths,
        'label': labels
    })

    # Show the distribution of labels in the dataset
    dataset_distribution_dict = {i: len(data[data['label'] == i]) for i in range(len(classes))}
    print(dataset_distribution_dict)
    print()

    return data


In [ ]:
# Define classes
classes = ['0', '1', '2', '3', '4']

# Load training and testing datasets
train_df = load_dataset_as_dataframe('train')
test_df = load_dataset_as_dataframe('test')


Load dataset from `train` subdirectory
Classes: ['0', '1', '2', '3', '4']
{0: 100, 1: 0, 2: 0, 3: 0, 4: 0}

Load dataset from `test` subdirectory
Classes: ['0', '1', '2', '3', '4']
{0: 100, 1: 0, 2: 0, 3: 0, 4: 0}



In [ ]:
# Define image transformations for training and testing
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet standards
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [ ]:
class KneeDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)  # Reset the index for consistency
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.loc[idx, 'image_path']
        label = self.df.loc[idx, 'label']
        image = Image.open(img_path).convert('RGB')  # Open image and convert to RGB
        if self.transform:
            image = self.transform(image)  # Apply transformations
        return image, label


In [ ]:
# Create dataset instances for training and testing
train_dataset = KneeDataset(train_df, transform=train_transform)
test_dataset = KneeDataset(test_df, transform=test_transform)


In [8]:
# Function to display images
def imshow(img):
    img = img.numpy().transpose((1, 2, 0))
    # Unnormalize
    mean = np.array([0.485, 0.456, 0.406])
    std  = np.array([0.229, 0.224, 0.225])
    img  = std * img + mean
    img  = np.clip(img, 0, 1)
    plt.imshow(img)
    plt.axis('off')

# Get a batch of training data
images, labels = next(iter(train_loader))

# Make a grid from batch
out = utils.make_grid(images)

# Display images
plt.figure(figsize=(10, 10))
imshow(out)
plt.show()

RuntimeError: DataLoader worker (pid(s) 27844, 13880) exited unexpectedly